<a href="https://colab.research.google.com/github/AchintyaX/Unsupervised_Sentiment_Analysis/blob/master/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the support libraries 

In [ ]:
import pandas as pd 
import numpy as np
import nltk 
import re 
from re import sub
import string 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import sys
sys.path.append('/content/drive/My Drive/jio_project/sentiment_analysis/Sentiment_demo')
import spacy
from spacy.lang.hi import Hindi
import numpy as np 
import textblob
from textblob import TextBlob 
from gensim.models.fasttext import FastText 
import en_core_web_sm 

# Importing all the Modules 
### Importing all the functions that have been developed for preprocessing 
### and for sentiment analysis

In [ ]:
from article_preprocessing import remove_named_entities, preprocess, tokenize_named_entities_removal, remove_stopwords, preprocess_hin, load_dicts
from model import sentiment_coeff, term_frequency, predict_sentiment, get_senti_coeff_indic, get_sentiment, word_gen

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Loading the semantic dependencies 

In [ ]:
POS_WORDS_PATH = '/content/drive/My Drive/jio_project/sentiment_analysis/Sentiment_demo/positive_words_hi.csv'
NEG_WORDS_PATH = '/content/drive/My Drive/jio_project/sentiment_analysis/Sentiment_demo/negative_words_hi.csv'

In [ ]:
pos_words_dict = load_dicts(POS_WORDS_PATH)
neg_words_dict = load_dicts(NEG_WORDS_PATH)

Loading the FastText Model 

In [ ]:
model = FastText.load_fasttext_format('/content/drive/My Drive/jio_project/sentiment_analysis/Sentiment_demo/cc.hi.300.bin')

## Word_gen
The word_gen function is used to generate new words of similar polarity from a given set of words. 
The arguments of word_gen are -
1. List of words 
2. Polarity of the list of words 
3. FastText model
4. postitve and negative reference words for polarity detection 

Here we are using word_gen on a few negative words to generate more negative words. All the words that are generated are different.

In [ ]:
word_array = ['असमर्थ', 'गंदा', 'मैला']
word_gen(word_array, model, -1, pos_words_dict, neg_words_dict)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['सक्षम',
 'समर्थ',
 'मजबूर',
 'विफल',
 'बाध्य',
 'गन्दा',
 'गंदे',
 'गंदगी',
 'हॉकर्सघरों',
 'बदबूदार',
 'बजबजाता',
 'बजबजाते',
 'बजबजाती',
 'कुचैला',
 'भजयि',
 'साढदेव',
 'पठानमुख',
 'ढ़ोने']

Here we are generating a few positive words to generate more positive words 

In [ ]:
word_array = ['उत्कृष्ट', 'गर्व', 'प्रभावशाली']
word_gen(word_array, model, 1, pos_words_dict, neg_words_dict)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['बेहतरीन', 'उमा०', 'शानदार', 'श्रेष्ठ', 'उत्तम', 'BSK']

## get_sentiment 

This function is used to get the polarity of the a given word token, 
The module with the help of the posiitve and negative words list is able to compare and give us the positive and negative polarity of a word,
Comparing the two one can find whether the word is positive or negative 
The arguments for the function are - 
1. target word
2. known polar words 
3. FastText language model

Here get_sentiment is used to get the negative polarity of a the word 'गंदगी'

In [ ]:
get_sentiment('गंदगी', neg_words_dict, model)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.7499999552965164

Here we are using get_sentiment to get the positive polarity of the word 'उत्कृष्ट'

In [ ]:
get_sentiment('उत्कृष्ट', pos_words_dict, model)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.42201653122901917

The get_similarity function is the alternative for libraries like TextBlob which provide lexicon polarity in english

In [ ]:
# there is already existing tool for lexicon polarity prediction for english 
print(TextBlob('awesome').polarity)
print(TextBlob('dirty').polarity)

1.0
-0.6


## get_senti_coeff_indic and senti_coeff

this function helps us generate a sentiment vector for a given article, it is basically finds the polarity of each word in the article and replaces them with it. 
arguments for get_senti_coeff are - 
1. article
2. positive reference words
3. negative reference words 
4. FastText language model

Let us take a sample news headline, we need to preprocess it a bit 

In [ ]:
article_hin = 'छपरा में व्यवसायी का शव बरामद'

#tokenizing hindi data 
def tokenize_hindi(article):
    nlp = Hindi()
    doc = nlp(article)
    tokens = [token.text for token in doc]
    
    return tokens 
article_hin = preprocess_hin(article_hin)
artcile_hin = tokenize_hindi(article_hin)

we can see that apart from 'शव' all the other words are neutral 

In [ ]:
get_senti_coeff_indic(article_hin, pos_words_dict, neg_words_dict, model)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.40277905762195587,
 0,
 0,
 0,
 0]

Similarly let us take an english news headline and find the sentiment vector

In [ ]:
article_eng = 'Loan demand in low-ticket segment robust but slowing for high-end market, says HDFCs Keki Mistry'

# Preprocessing the article
article_eng = remove_named_entities(article_eng)
article_eng = preprocess(article_eng)
article_eng = tokenize_named_entities_removal(article_eng)
article_eng = remove_stopwords(article_eng)

as we can see respective sentiment coeff has been given to each token 

In [ ]:
sentiment_coeff(article_eng)

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 -0.30000000000000004,
 0.16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

## Term_frequency 

This function generates a vector that represents an article on the basis of frequency of occurence of a word in the article. 
This function is common and doesn't require any language specific input 

In [ ]:
term_frequency(article_hin)

[0.034482758620689655,
 0.034482758620689655,
 0.06896551724137931,
 0.13793103448275862,
 0.1724137931034483,
 0.06896551724137931,
 0.034482758620689655,
 0.034482758620689655,
 0.1724137931034483,
 0.10344827586206896,
 0.034482758620689655,
 0.06896551724137931,
 0.10344827586206896,
 0.034482758620689655,
 0.13793103448275862,
 0.06896551724137931,
 0.034482758620689655,
 0.1724137931034483,
 0.034482758620689655,
 0.13793103448275862,
 0.1724137931034483,
 0.034482758620689655,
 0.10344827586206896,
 0.1724137931034483,
 0.034482758620689655,
 0.06896551724137931,
 0.13793103448275862,
 0.06896551724137931,
 0.034482758620689655]

# Predict_sentiment
This is the final function it can be used for any langauge just language code has to be passed on along side the article.
<br>
The arguments for Predict_sentiment are -
1. article
2. langauge_code
3. fasttext language model
4. positive reference words
5. negative reference words 

Here we are showing an example of it at work with the given hindi news headline<br>- 
<i>"छपरा में व्यवसायी का शव बरामद"</i> <br>

as we can see that the sentiment is negative, if the model gives output -1 then it means it interprets the sentiment correctly

In [ ]:
predict_sentiment(article_hin, 'hi', model, pos_words_dict, neg_words_dict)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


-1

Here we are showing an exmple of the function at work for a given english news headline - <br>
-<i>"Loan demand in low-ticket segment robust but slowing for high-end market, says HDFCs Keki Mistry " </i>

as we can see the article the general sentiment of the article is negative so, if the model interprets it correctly it would return <br> -1

In [ ]:
predict_sentiment(article_eng, 'en', model, pos_words_dict, neg_words_dict)

-1